In [3]:
#!/usr/bin/env python
# coding: utf-8
from data_chooser import interactive_data_chooser, create_fake_df


from dash import Dash, dcc, html, Input, Output
import plotly.express as px


df = create_fake_df(500)

chooser = interactive_data_chooser(df, df.columns)
chooser.activate_plot()


Selected 17 new points. Total: 17
Selected 6 new points. Total: 23
Selected 5 new points. Total: 28


In [7]:
chooser.outlier_df

,x,y1,y2
0,9,52,12
1,10,72,63
2,11,48,35
3,13,64,79
4,13,44,73
5,12,41,5
6,9,73,78
7,14,64,61
8,9,72,96
9,12,67,54
